In [1]:
# Import libraries

import sys
import pypsa
import logging
import pandas as pd
import numpy as np
import xarray as xr
from plotly import graph_objects as go
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from model.assumptions_core import read_assumptions
from input.library.weather import load_weather
from model.constraints_core import calculate_biogas_max

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    "EUR": 10.5080,
    "USD": 9.2521
}

base_year = 2024
discount_rate = 0.04
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3
biogas_method = "average"

## Parameters that will change frequently
geography = '14'
target_year = 2030
self_sufficiency = 0.8
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 0.25
growth_only = True

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_root / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the demand from csv file
projected_demand_path = paths.demand / f"projected-demand,geography={geography.replace(':','-')},target-year={target_year},growth-only={growth_only}.csv.gz"
demand = pd.read_csv(projected_demand_path, compression='gzip', index_col='timestamp')
target_load = demand['value'].values.flatten()

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
weather_geo = geography.split(':',1)[0] # All of VGR
section = None
cutout, selection, index = load_weather(weather_geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_root / 'renewables' / f"capacity-factor-{weather_geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_root / 'renewables' / f"capacity-factor-{weather_geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_root / 'renewables' / f"capacity-factor-{weather_geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [4]:
# Build the network

def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

def annualized_capex(asset):
    return (annuity(discount_rate, float(assumptions.loc[(asset, 'lifetime'), 'value'])) + float(assumptions.loc[(asset, 'FOM'), 'value'])) * float(assumptions.loc[(asset, 'capital_cost'), 'value'])

## Initialize the network
network = pypsa.Network()
network.set_snapshots(index)
network.snapshot_weightings.loc[:, :] = resolution

## Carriers
carriers = [
    'AC',
    'onwind',
    'offwind',
    'solar',
    'li-ion',
    'h2',
    'biogas',
    'mixedgas',
    'backstop',
    'import',
    ]

carrier_colors = ['black', 'green', 'blue', 'red', 'lightblue', 'grey', 'brown', 'brown', 'white', 'white']

network.madd(
    'Carrier',
    carriers,
    color=carrier_colors,
    )

## Load bus location
minx, miny, maxx, maxy = selection.total_bounds
midx = (minx + maxx)/2
midy = (miny + maxy)/2

## Add the buses
network.add('Bus', 'load-bus', carrier='AC', x=midx, y=midy)
network.add('Bus', 'renewables-bus', x=midx+0.5, y=midy+0.25)
network.add('Bus', 'battery-bus', carrier='li-ion', x=midx-0.5, y=midy)
if use_h2 or biogas_limit > 0:
    network.add('Bus', 'turbine-bus', x=midx, y=midy+0.5)
if use_h2:
    network.add('Bus', 'h2-bus', carrier='h2', x=midx-0.5, y=midy+0.5)

## Add load and backstop to load bus
network.add('Load', 'load', bus='load-bus',
            p_set=target_load
            )

network.add('Generator', 'backstop', carrier='backstop', bus='load-bus',
            p_nom_extendable=True,
            capital_cost=assumptions.loc[('backstop', 'capital_cost'), 'value'],
            marginal_cost=assumptions.loc[('backstop', 'marginal_cost'), 'value'],
            lifetime=assumptions.loc[('backstop', 'lifetime'), 'value'],
            )

network.add('Generator', 'market', carrier='import', bus='load-bus',
            p_nom_extendable=True,
            capital_cost=0,
            marginal_cost=600,
            lifetime=100
            )

## Add generators and links to renewable bus

network.add('Generator', 'solar', carrier='solar', bus='renewables-bus',
            p_nom_extendable=True, 
            p_max_pu=capacity_factor_solar,
            p_nom_mod=assumptions.loc['solar','unit_size'].value,
            capital_cost= annualized_capex('solar'),
            marginal_cost=assumptions.loc[('solar', 'VOM'), 'value'],
            lifetime=assumptions.loc[('solar', 'lifetime'), 'value'],
            )

network.add('Generator', 'onwind', carrier='onwind', bus='renewables-bus',
            p_nom_extendable=True,
            p_max_pu=capacity_factor_onwind,
            p_nom_mod=assumptions.loc['onwind','unit_size'].value,
            capital_cost= annualized_capex('onwind'),
            marginal_cost=assumptions.loc[('onwind', 'VOM'), 'value'],
            lifetime=assumptions.loc['onwind','lifetime'].value,
            )

if use_offwind:
    network.add('Generator', 'offwind', carrier='offwind', bus='renewables-bus',
                p_nom_extendable=True,
                p_max_pu=capacity_factor_offwind,
                p_nom_mod=assumptions.loc['offwind','unit_size'].value,
                capital_cost= annualized_capex('offwind'),
                marginal_cost=assumptions.loc[('offwind', 'VOM'), 'value'],
                lifetime=assumptions.loc['offwind','lifetime'].value,
                )

network.add('Link', 'Renewables load link', bus0='renewables-bus', bus1='load-bus',
            p_nom_extendable=use_offwind,
            )

## Add battery storage

network.add('Link','battery-charge', bus0='renewables-bus', bus1='battery-bus',
            p_nom_extendable = True,
            capital_cost= annualized_capex('battery_inverter'),
            marginal_cost=assumptions.loc['battery_inverter','VOM'].value,
            lifetime=assumptions.loc['battery_inverter','lifetime'].value,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

network.add('Store', 'battery', carrier='li-ion', bus='battery-bus',
            e_initial=100,
            e_nom_extendable=True,
            e_cyclic=True,
            e_min_pu=0.15,
            standing_loss=0.00008, # TODO: Check if this is really per hour as in the documentation or if it is per snapshot
            capital_cost= annualized_capex('battery_storage'),
            marginal_cost=assumptions.loc['battery_storage','VOM'].value,
            lifetime=assumptions.loc['battery_storage', 'lifetime'].value,
            )

network.add('Link','battery-discharge', carrier='li-ion', bus0='battery-bus', bus1='load-bus',
            p_nom_extendable = True,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

## Add H2 electrolysis, storage, pipline to gas turbine

if use_h2:
    network.add('Link', 'h2-electrolysis', carrier='h2', bus0='renewables-bus', bus1='h2-bus',
                p_nom_extendable=True,
                p_nom_mod=assumptions.loc['h2_electrolysis','unit_size'].value,
                capital_cost= annualized_capex('h2_electrolysis'),
                marginal_cost=assumptions.loc[('h2_electrolysis', 'VOM'), 'value'],
                lifetime=assumptions.loc['h2_electrolysis','lifetime'].value,
                efficiency=assumptions.loc['h2_electrolysis','efficiency'].value,
                )

    network.add('Store', 'h2', carrier='h2', bus='h2-bus',
                e_initial=(150_000 if use_h2 else 0),
                e_nom_extendable=use_h2,
                e_cyclic=True,
                capital_cost= annualized_capex('h2_storage'),
                marginal_cost=assumptions.loc['h2_storage','VOM'].value,
                lifetime=assumptions.loc['h2_storage','lifetime'].value
                )

    network.add('Link', 'H2 pipeline', carrier='h2', bus0='h2-bus', bus1='turbine-bus',
                p_nom_extendable=True,
                )

### Biogas pipeline

if biogas_limit > 0:
    network.add('Generator', 'biogas-market', carrier='biogas', bus='turbine-bus',
                p_nom_extendable=True,
                p_nom_max=biogas_max(biogas_limit, target_load, assumptions.loc['combined_cycle_gas_turbine','efficiency'].value, "average"),
                marginal_cost=assumptions.loc['biogas','cost'].value,
                lifetime=100,
                )

### Gas turbine
if use_h2 or biogas_limit > 0:
         
    network.add('Link', 'gas-turbine', carrier='mixedgas', bus0='turbine-bus', bus1='load-bus',
                p_nom_extendable=True,
                p_nom_mod=assumptions.loc['combined_cycle_gas_turbine','unit_size'].value,
                capital_cost= annualized_capex('combined_cycle_gas_turbine'),
                marginal_cost=assumptions.loc['combined_cycle_gas_turbine','VOM'].value,
                lifetime=assumptions.loc['combined_cycle_gas_turbine','lifetime'].value,
                efficiency=assumptions.loc['combined_cycle_gas_turbine','efficiency'].value,
                )

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Add self-sufficiency constraint on the import market
market_e = model.variables['Generator-p'].loc[:,'market'].sum()
non_sufficiency_e = target_load.sum() * (1 - self_sufficiency)
model.add_constraints(market_e <= non_sufficiency_e, name="Self_sufficiency_constraint")

## Add offwind constraint
if False:
    offwind_percentage = 0.5
    offwind_e = model.variables['Generator-p'].loc[:,'offwind'].sum()
    onwind_e = model.variables['Generator-p'].loc[:,'onwind'].sum()

    #offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    offwind_constraint = offwind_e - offwind_percentage * (onwind_e + offwind_e)
    model.add_constraints(offwind_constraint >= 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
link_capacity = model.variables["Link-p_nom"]
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 670.66it/s]
INFO:linopy.io: Writing time: 0.5s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 46739 primals, 102222 duals
Objective: 7.60e+09
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance, Self_sufficiency_constraint were not assigned to the network.


Coefficient ranges:
  Matrix [4e-06, 3e+02]
  Cost   [6e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [7e+02, 1e+06]
Presolving model
48185 rows, 45272 cols, 132872 nonzeros  0s
38760 rows, 35847 cols, 120918 nonzeros  0s
38038 rows, 35125 cols, 123282 nonzeros  0s

Solving MIP model with:
   38038 rows
   35125 cols (0 binary, 5 integer, 0 implied int., 35120 continuous)
   123282 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.2s
 S       0       0         0   0.00%   -inf            7600856489.547     Large        0      0      0         0    38.7s
 R       0       0         0   0.00%   7596634341.358  7596693420.623     0.00%        0      0      0     61627    4

('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator biogas        7.457878e+02                 0.0  6.356291e-15   
          market        2.545502e+03                 0.0  3.000000e+06   
          onwind        3.933977e+03                 0.0  8.058029e+06   
          solar         3.821999e+03                 0.0  4.013944e+06   
Link      AC            3.313965e+03                 0.0  1.203582e+07   
          h2            1.184238e-15                 0.0  5.177402e-12   
          li-ion        6.762355e+02                 0.0  8.133124e+05   
          mixedgas      0.000000e+00                 0.0  5.015906e-12   
Load      -             0.000000e+00                 0.0  0.000000e+00   
Store     h2            1.589073e-15                 0.0  1.495588e-13   
          li-ion        4.725454e+03                 0.0  7.594368e+05   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator biogas    7.706048e-14 -7.070419e-14  0.000000e+00     1.276833e-20   
          market    0.000000e+00  3.000000e+06  0.000000e+00     1.345376e-01   
          onwind    0.000000e+00  8.058029e+06  0.000000e+00     2.338261e-01   
          solar     0.000000e+00  4.013944e+06  0.000000e+00     1.198883e-01   
Link      AC        1.207197e+07 -3.614975e+04  1.118669e+07     4.158399e-01   
          h2        5.160954e-12  1.644812e-14  0.000000e+00     4.998582e-01   
          li-ion    8.479371e+05 -3.462466e+04  0.000000e+00     1.431402e-01   
          mixedgas  2.899626e-12  2.116280e-12 -4.967794e-12              NaN   
Load      -         1.500000e+07 -1.500000e+07  0.000000e+00              NaN   
Store     h2        6.874251e-12 -6.724693e-12  0.000000e+00     3.077484e+00   
          li-ion    7.606354e+05 -1.198672e+03  0.000000e+00     3.619908e-01   

                     Curtailment  Capital Expenditure  \
Generator biogas    0.000000e+00         0.000000e+00   
          market    0.000000e+00         0.000000e+00   
          onwind    5.131849e+06         3.581432e+09   
          solar     3.036591e+05         1.338288e+09   
Link      AC        0.000000e+00         9.286047e+07   
          h2        0.000000e+00         0.000000e+00   
          li-ion    0.000000e+00         0.000000e+00   
          mixedgas  0.000000e+00         0.000000e+00   
Load      -         0.000000e+00         0.000000e+00   
Store     h2        0.000000e+00         6.446363e-11   
          li-ion    0.000000e+00         6.124925e+08   

                    Operational Expenditure  Revenue  Market Value  
Generator biogas              -4.086278e-11      0.0           NaN  
          market               1.800000e+09      0.0           NaN  
          onwind               1.716458e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          h2                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
          mixedgas            -2.366297e-10      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     h2                   0.000000e+00      0.0           NaN  
          li-ion              -2.488900e+04      0.0           NaN